In [ ]:
import requests
import time
import random
from bs4 import BeautifulSoup
import pandas as pd

base = "https://movie.naver.com/movie/point/af/list.nhn?&page={}"
clist = []
for page in range(1, 1001): #네이버 영화 평점 페이지는 1000페이지까지만 조회가 가능
    url = base.format(page)
    res = requests.get(url)
    if res.status_code == 200:
        soup = BeautifulSoup(res.text, "lxml")
        tds = soup.select("table.list_netizen > tbody > tr > td.title")
        for td in tds:
            title = td.select_one("a.movie").text.strip()
            score = td.select_one("div.list_netizen_score > em").text.strip()
            comment = td.select_one("br").next_sibling.strip()
            clist.append((title, score, comment))
        interval = round(random.uniform(0.2, 1.2), 2)
        time.sleep(interval)
    print("종료")
df1 = pd.DataFrame(clist, columns = ["title", "rating", "comment"])

In [3]:
df1.head()

,title,rating,comment
0,체르노빌 1986,8,실화영화 잘 봤어요~~
1,인 더 하이츠,8,노래 네곡이 딱 좋았어요In the HeightsBreathe할머니노래(제목이?)C...
2,블랙 위도우,10,마블영화중에 제일재미있게봤어요
3,발신제한,10,결말이 허무하다는 말이 많았는데스릴러 좋아하는 사람으로서 너무 재밌었고긴장감도 너무...
4,블랙 위도우,4,윈터솔져에서 액션 양 절반 줄이고 질 절반 줄이면 블랙위도우인듯 음악은 완전 좋았어요


In [26]:
#rating & comment가 10개 이하인 영화는 제외
idx = df1["title"].value_counts().reset_index().query("title < 10")["index"]
df2 = df1[df1["title"].isin(idx) == False]
df2["title"].value_counts()

제8일의 밤         1986
미드나이트          1123
발신제한            630
크루엘라            508
콰이어트 플레이스 2     360
               ... 
크롤               10
콰이어트 플레이스        10
걸캅스              10
클래식              10
비와 당신의 이야기       10
Name: title, Length: 68, dtype: int64

In [63]:
df1.to_csv("nmovie.csv", encoding = "utf-8")